In [21]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import urllib
import csv
import numpy as np

headers = { 'User-Agent': 'Mozilla/5.0 (Windows NT 6.0; WOW64; rv:24.0) Gecko/20100101 Firefox/24.0' }

In [3]:
wiki_artists = pd.read_excel('../data/wiki_house_musicians.xlsx')

In [4]:
discogs_artists = pd.read_excel('../data/discogs_artists.xlsx')

In [7]:
wiki_artists.columns

Index([u'artist_name'], dtype='object')

In [8]:
discogs_artists.columns

Index([u'artist_name'], dtype='object')

In [10]:
artists = wiki_artists.append(discogs_artists)

In [15]:
artists.drop_duplicates(inplace=True)

In [16]:
len(artists)

9426

In [17]:
artist = 'disclosure band name'

In [22]:
artist = 'Calvin Harris'

In [34]:
url = 'https://www.google.com/search?sclient=psy-ab&client=ubuntu&hs=k5b&channel=fs&biw=1366&bih=648&noj=1&q=' + '+'.join(artist.split()) + '+soundcloud+page'
r = requests.get(url, headers = headers)
html = r.content
soup = BeautifulSoup(html, 'html.parser')
print soup.find('cite').text

https://soundcloud.com/calvinharris


In [42]:
res = soup.find('cite').text

In [40]:
res = 'fjdkslajfsa'

In [43]:
res.split('/')

[u'https:', u'', u'soundcloud.com', u'calvinharris']

In [44]:
len(res.split('/'))

4

In [45]:
import psycopg2
from datetime import datetime

In [48]:
conn = psycopg2.connect(dbname='soundcloud', user='postgres', host='/tmp')

In [49]:
c = conn.cursor()

### Create a table

In [105]:
query = """
CREATE TABLE sc_aliases (
    id SERIAL PRIMARY KEY
    ,artist_name VARCHAR
    ,sc_alias VARCHAR
);
"""

In [106]:
c.execute(query)

In [107]:
conn.commit()

### Add some rows to the table

In [108]:
query = """
INSERT INTO
    sc_aliases
    (artist_name, sc_alias)
    VALUES (%s, %s)
"""

In [113]:
def get_SC_alias(artist):
    try:
        url = 'https://www.google.com/search?sclient=psy-ab&client=ubuntu&hs=k5b&channel=fs&biw=1366&bih=648&noj=1&q=' + '+'.join(artist.split()) + '+soundcloud+page'
        r = requests.get(url, headers = headers)
        html = r.content
        soup = BeautifulSoup(html, 'html.parser')
        res = soup.find('cite').text
        res = res.split('/')
        alias = 'No Soundcloud Found'
        if 'https://soundcloud.com/' in res and len(res) == 4:
            alias = res[3]
            c.execute(query, (artist, alias))
            conn.commit()
        else:
            c.execute(query, (artist, 'No Soundcloud Found'))
        return alias
    except Exception as e:
        return str(e)

In [114]:
artists['sc_alias'] = 

,artist_name
6,009 Sound System
7,2 Unlimited
8,49ers
9,808 State
10,A-Trak


In [95]:
c.fetchall()

[(1, 'test_name', 'test_alias')]

In [103]:
#c.execute("DROP TABLE sc_aliases")